In [ ]:
# /*==========================================================================================*\
# **                        _           _ _   _     _  _         _                            **
# **                       | |__  _   _/ | |_| |__ | || |  _ __ | |__                         **
# **                       | '_ \| | | | | __| '_ \| || |_| '_ \| '_ \                        **
# **                       | |_) | |_| | | |_| | | |__   _| | | | | | |                       **
# **                       |_.__/ \__,_|_|\__|_| |_|  |_| |_| |_|_| |_|                       **
# \*==========================================================================================*/


# Author: Bùi Tiến Thành (@bu1th4nh)
# Date: 2022/12/26 14:13 
# CTTN Toán tin K64

from tqdm import tqdm
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')
import statsmodels.api as sm
%matplotlib inline
import os
import re
import ujson as json
import os
import ot
from dtaidistance import dtw

# Figure size
height = 20
width  = 8

## Input, Resample and Normalization

In [ ]:
Ariel = pd.read_parquet("../DataWater_train_cleansed_phase3.parquet")

In [ ]:
# Ariel.head()
Belle = Ariel.copy().resample('D').mean().reset_index(drop=True)

In [ ]:
attributes  = Ariel.columns
mean        = Ariel.describe().loc["mean", :].to_numpy()
std         = Ariel.describe().loc["std", :].to_numpy()
D           = len(attributes)
T           = len(Ariel)

print("Mean: ", mean);
print("Std:  ", std);

In [ ]:
# Do data được đo theo giờ nên ta có thể chuyển qua thời gian tương đối
initial_time = Ariel.index[0];
Ariel.reset_index(drop=True, inplace=True)
for i, col in enumerate(Ariel.columns):
    Ariel[col] = Ariel[col].apply(lambda x: (x - mean[i]) / std[i])

timestamp = range(Ariel.shape[0])

## Xây dựng ma trận đánh dấu

In [ ]:
mask = ~pd.isnull(Ariel)
display(mask.head(5))

## Xây dựng ma trận delta dựa theo lags


In [ ]:
# Số lượng mốc thời gian dùng để dự báo
n_steps  = 96;

# Danh sách các bộ (start, end) với data lấy từ start->end-1 và data dự báo là end
idx_list = [];

for i in tqdm(range(T - n_steps), desc="Building delta and samples"):
    # Lấy khoảng thời gian
    (start, end) = (i, i + n_steps)

    # Xây dựng delta
    delta = np.zeros((n_steps, D))
    for i in range(1, n_steps):
        delta[i, :] = timestamp[start+i] - timestamp[start+i-1] + delta[i-1, :] * mask.iloc[start+i-1, :].astype(int).to_numpy();
    delta = pd.DataFrame(delta, columns=attributes)

    # Thêm vào danh sách
    idx_list.append((start, end, delta))

## Xây dựng ma trận HSTQ

In [ ]:
p_list = [0, 0.0001, 0.001, 0.01, 0.1, 0.5, 1.0, 10, 100]
for epoch in range(len(p_list)):
    df_null = None
    df_dtw = None
    df_dtw_f = None
    penalty = p_list[epoch]
    print('p:{}'.format(penalty))
    w = None
    length = n_steps

    for i in tqdm(range(len(idx_list)), desc=f"Processing idx_list with p={penalty}"):
        # if i % 20 == 0:
        #     print('Processing {} of {}'.format(i, len(idx_list)))

        (start, end, df_deltas) = idx_list[i]
        df_mask                 = mask.loc[start:end]
        df_values               = Ariel.loc[start:end].copy()
        



        df_values[~df_mask.astype(bool)] = np.nan
        df_values.interpolate(method='linear', axis=0, inplace=True)
        df_values.fillna(0, inplace=True)
        distance = dtw.distance_matrix_fast(df_values.to_numpy().astype(float).swapaxes(1,0))


        num = df_mask.sum()
        miss_num = (df_deltas * (1-df_mask)).sum()
        miss_num = np.tile(miss_num, (35,1))
        penalty_matrix = miss_num + miss_num.transpose()


        distance += penalty_matrix * penalty

        num = np.tile(num, (35,1))
        w_matrix = (num + num.transpose())/(2*length)


        if df_dtw is not None:
            df_dtw += distance * w_matrix
            w += w_matrix
        else:
            df_dtw = distance * w_matrix
            w = w_matrix
            

    inf_idx = np.isinf(df_dtw)
    df_dtw[inf_idx] = 0
    nan_idx = np.isnan(df_dtw)
    df_dtw[nan_idx] = 0

    df_dtw = df_dtw/(w+1e-5)

    df_dtw += df_dtw.transpose()
    df_dtw[np.where(np.eye(df_dtw.shape[0],dtype=bool))] += np.min(df_dtw[np.where(~np.eye(df_dtw.shape[0],dtype=bool))])



    def normalization(data):
        _range = np.max(data) - np.min(data)
        return (data - np.min(data)) / _range


    # plt.figure()
    # sns.heatmap(abs(w), cmap='Reds')
    # plt.close()

    # df_dtw_exp = normalization(np.exp(-df_dtw))
    # plt.figure()
    # sns.heatmap(abs(df_dtw_exp), cmap='Reds')
    # plt.savefig('../../../docs/figures/LPDTW/simple/p_'+str(penalty)+'.png')
    # plt.close()

    df_dtw_inv = normalization(1/df_dtw)
    column_names = [i for i in range(1,36)]
    df_dtw_inv = pd.DataFrame(df_dtw_inv, columns=column_names, index=column_names)
    plt.figure()
    sns.heatmap(abs(df_dtw_inv), cmap='Reds')


    # plt.savefig('../../../docs/figures/POT/p_inv_' + str(penalty) + '.eps', format='eps', dpi=300, bbox_inches='tight')
    # plt.savefig('figures/PDTW/p_' + str(penalty) + '.png', format='png', bbox_inches='tight')
    # plt.close()

    # np.savetxt('../data/POT_exp_' + str(penalty) + '.csv', df_ot_exp.to_numpy(), delimiter=',')
    # np.savetxt('matrix/PDTW_' + str(penalty) + '.csv', df_dtw_inv.to_numpy(), delimiter=',')


print('Done! Figures are saved to \'{}\', Correlation Matrices are saved to \'{}\''.format('figures/PDTW/','matrix/'))